# Notebook 1: Extract Z-scored LFP

- Notebook for extracting local field potential traces from Spikegadgets `.rec` files

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [10]:
# NOTE: Change based on individual project data location

# Spreadsheet of channel mapping
CHANNEL_MAPPING_DF = pd.read_excel("../../data/channel_mapping.xlsx")
# Spreadsheet of tone time
TONE_TIMESTAMP_DF = pd.read_csv("../../proc/rce_tone_timestamps.csv", index_col=0)

In [11]:
CHANNEL_MAPPING_DF.head()

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0


In [12]:
TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,video_name,baseline_lfp_index_range,trial_lfp_index_range,baseline_ephys_index_range,trial_ephys_index_range,baseline_videoframe_range,trial_videoframe_range,all_subjects,current_subject,trial_outcome
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,20221202_134600_omission_and_competition_subje...,"(59541, 69541)","(69541, 79541)","(1190826, 1390826)","(1390826, 1590826)","(1514, 1734)","(1734, 1954)","('6.1', '6.2')",6.1,rewarded
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,20221202_134600_omission_and_competition_subje...,"(139541, 149541)","(149541, 159541)","(2790825, 2990825)","(2990825, 3190825)","(3508, 3728)","(3728, 3948)","('6.1', '6.2')",6.1,rewarded
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,20221202_134600_omission_and_competition_subje...,"(229541, 239541)","(239541, 249541)","(4590823, 4790823)","(4790823, 4990823)","(5752, 5972)","(5972, 6192)","('6.1', '6.2')",6.1,rewarded
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,20221202_134600_omission_and_competition_subje...,"(309541, 319541)","(319541, 329541)","(6190821, 6390821)","(6390821, 6590821)","(7746, 7966)","(7966, 8186)","('6.1', '6.2')",6.1,omission
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,20221202_134600_omission_and_competition_subje...,"(384541, 394541)","(394541, 404541)","(7690820, 7890820)","(7890820, 8090820)","(9616, 9836)","(9836, 10056)","('6.1', '6.2')",6.1,rewarded


In [13]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/*/*.rec")

In [14]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115641_standard_comp_to_omission_D1_subj_2-2_and_2-4.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_02/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_03/20221203_154800_omission_and_competition_subject_6_4_and_6_1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100646_standard_comp_to_omission_D2_subj_2-4_and_2-1.rec',
 '/scratch/back

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"../../proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [16]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Extracting the LFP

In [17]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero
    




An exception occurred: stream_id trodes is not in ['ECU']
20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged
20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20221202_134600_omission_and_competition_subject_6_1_top_2_base_3_merged
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20221203_154800_omission_and_competition_subject_6_1_top_1_base_3_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
20230618_10063

- Filtering for all trials that we got the LFP for

In [18]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF[TONE_TIMESTAMP_DF["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [19]:
TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,video_name,baseline_lfp_index_range,trial_lfp_index_range,baseline_ephys_index_range,trial_ephys_index_range,baseline_videoframe_range,trial_videoframe_range,all_subjects,current_subject,trial_outcome
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,20221202_134600_omission_and_competition_subje...,"(59541, 69541)","(69541, 79541)","(1190826, 1390826)","(1390826, 1590826)","(1514, 1734)","(1734, 1954)","('6.1', '6.2')",6.1,rewarded
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,20221202_134600_omission_and_competition_subje...,"(139541, 149541)","(149541, 159541)","(2790825, 2990825)","(2990825, 3190825)","(3508, 3728)","(3728, 3948)","('6.1', '6.2')",6.1,rewarded
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,20221202_134600_omission_and_competition_subje...,"(229541, 239541)","(239541, 249541)","(4590823, 4790823)","(4790823, 4990823)","(5752, 5972)","(5972, 6192)","('6.1', '6.2')",6.1,rewarded
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,20221202_134600_omission_and_competition_subje...,"(309541, 319541)","(319541, 329541)","(6190821, 6390821)","(6390821, 6590821)","(7746, 7966)","(7966, 8186)","('6.1', '6.2')",6.1,omission
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,20221202_134600_omission_and_competition_subje...,"(384541, 394541)","(394541, 404541)","(7690820, 7890820)","(7890820, 8090820)","(9616, 9836)","(9836, 10056)","('6.1', '6.2')",6.1,rewarded


- Adding trial numbers based on timestamp ordering for each recording

In [21]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.groupby('recording_file').apply(lambda x: helper.compute_sorted_index(x, value_column='time', index_column='trial_number')).reset_index(drop=True)

In [22]:
TONE_TIMESTAMP_DF["trial_number"] = TONE_TIMESTAMP_DF["trial_number"] + 1

## Adding the LFP trace information

In [23]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


- Adding all the brain region to ch information

In [24]:
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

In [27]:
TONE_TIMESTAMP_DF["current_subject"] = TONE_TIMESTAMP_DF["current_subject"].astype(str)

In [28]:
channel_map_and_TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

In [29]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=[col for col in channel_map_and_TONE_TIMESTAMP_DF.columns if "eib" in col], errors="ignore")

In [30]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=["Subject"], errors="ignore")

In [31]:
channel_map_and_TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,...,all_subjects,current_subject,trial_outcome,trial_number,Cohort,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,...,"('6.1', '6.2')",6.1,rewarded,1,1,21.0,15.0,14.0,13.0,16.0
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,...,"('6.1', '6.2')",6.1,rewarded,2,1,21.0,15.0,14.0,13.0,16.0
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,...,"('6.1', '6.2')",6.1,rewarded,3,1,21.0,15.0,14.0,13.0,16.0
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,...,"('6.1', '6.2')",6.1,omission,4,1,21.0,15.0,14.0,13.0,16.0
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,...,"('6.1', '6.2')",6.1,rewarded,5,1,21.0,15.0,14.0,13.0,16.0


In [32]:
channel_map_and_TONE_TIMESTAMP_DF.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'lfp_index', 'video_name',
       'baseline_lfp_index_range', 'trial_lfp_index_range',
       'baseline_ephys_index_range', 'trial_ephys_index_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'all_subjects',
       'current_subject', 'trial_outcome', 'trial_number', 'Cohort',
       'spike_interface_mPFC', 'spike_interface_vHPC', 'spike_interface_BLA',
       'spike_interface_LH', 'spike_interface_MD'],
      dtype='object')

In [33]:
raise ValueError()

ValueError: 

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.to_csv("./proc/trial_metadata.csv")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.to_pickle("./proc/trial_metadata.pkl")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.columns

- Linking up all LFP calculations with all the trials

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF["all_ch_lfp"] = channel_map_and_TONE_TIMESTAMP_DF["recording_file"].map(recording_name_to_all_ch_lfp)

In [ ]:
raise ValueError()

- Creating a new row for each brain region

In [ ]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [ ]:
id_cols = [col for col in channel_map_and_TONE_TIMESTAMP_DF.columns if col not in brain_region_col]

In [ ]:
brain_region_col

In [ ]:
for col in brain_region_col:
    channel_map_and_TONE_TIMESTAMP_DF[col] = channel_map_and_TONE_TIMESTAMP_DF[col].astype(int).astype(str)

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.columns

- Extracting the traces for each trial and brain region

In [ ]:
for col in brain_region_col:
    print(col)
    brain_region = col.strip("spike_interface").strip("_")
    baseline_trace_column = "{}_baseline_lfp_trace".format(brain_region)
    trial_trace_column = "{}_trial_lfp_trace".format(brain_region)
    baselineandtrial_trace_column = "{}_baseline-trial_lfp_trace".format(brain_region)
    
    channel_map_and_TONE_TIMESTAMP_DF[baseline_trace_column] = channel_map_and_TONE_TIMESTAMP_DF.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["baseline_lfp_timestamp_range"][0], end_frame=row["baseline_lfp_timestamp_range"][1]).T[0], axis=1)

    channel_map_and_TONE_TIMESTAMP_DF[trial_trace_column] = channel_map_and_TONE_TIMESTAMP_DF.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["trial_lfp_timestamp_range"][0], end_frame=row["trial_lfp_timestamp_range"][1]).T[0], axis=1)

    channel_map_and_TONE_TIMESTAMP_DF[baselineandtrial_trace_column] = channel_map_and_TONE_TIMESTAMP_DF.apply(lambda row: np.concatenate([row[baseline_trace_column], row[trial_trace_column]]), axis=1)                                                                                                              
                                                                                                                                                           
                                                                                                                                                    

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=["all_ch_lfp"], errors="ignore")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=[col for col in channel_map_and_TONE_TIMESTAMP_DF if "spike_interface" in col], errors="ignore")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.head()

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.columns

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.to_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")